# Student Id :ChinmayNandkishor_Mathakari@student.uml.edu(02007302) 

# Import libraries

In [2]:


import re
import os
import sys
import html
import spacy
import pandas as pd
import matplotlib.pyplot as plt


from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

# Q1.) Clean and remove short bug reports as in TextClassification.ipynb. Organize the remaining data to a data frame called df.

In [3]:
file = "eclipse_jdt.csv"
df = pd.read_csv(file)
df = df[['Issue_id','Priority','Component','Title','Description']]
df.head(5)

,Issue_id,Priority,Component,Title,Description
0,1518,P1,Debug,Icons needed for actions (1GI5UXW),JGS (8/8/01 5:20:19 PM); \tWe need enabled; di...
1,1519,P3,Debug,README: Hit count not reset (1GET20Y),JGS (6/4/01 11:43:47 AM); \tSet a breakpoint i...
2,1520,P3,Debug,Use styled text in console (1G9S1YF),DW (2/26/01 5:56:36 PM); \tCould allow users t...
3,1521,P3,Debug,StringBuffer representation (1GE3BFA),JGS (5/21/01 2:09:10 PM); \tTo be consistent; ...
4,1522,P2,Debug,Feature: use #toString to display variable val...,DW (9/27/00 11:12:31 AM); \tCurrently; the typ...


In [4]:
df = df[['Title','Description','Priority']]
df = df.dropna()
df['text'] = df['Title'] + ' ' + df['Description']
df = df.drop(columns=['Title','Description'])
df.head(5)

,Priority,text
0,P1,Icons needed for actions (1GI5UXW) JGS (8/8/01...
1,P3,README: Hit count not reset (1GET20Y) JGS (6/4...
2,P3,Use styled text in console (1G9S1YF) DW (2/26/...
3,P3,StringBuffer representation (1GE3BFA) JGS (5/2...
4,P2,Feature: use #toString to display variable val...


In [5]:
# Method defined to clean text 
def clean(text):
    text = html.unescape(text) # convert html escapes like &amp; to characters.
    text = re.sub(r'<[^<>]*>', ' ', text) # tags like <tab>
    text = re.sub(r'\[([^\[\]]*)\]\([^\(\)]*\)', r'\1', text) # markdown URLs like [Some text](https://....)
    text = re.sub(r'\[[^\[\]]*\]', ' ', text) # text or code in brackets like [0]
    text = re.sub(r'(?:^|\s)[&#<>{}\[\]+|\\:-]{1,}(?:\s|$)', ' ', text) # standalone sequences of specials, matches &# but not #cool
    text = re.sub(r'(?:^|\s)[\-=\+]{2,}(?:\s|$)', ' ', text) # standalone sequences of hyphens like --- or ==
    text = re.sub('[^A-Za-z]+', ' ', text) # remove any characters other than alphabets
    text = re.sub(r'\s+', ' ', text) # sequences of white spaces
    return text.strip()


df['text'] = df['text'].apply(clean)
df = df[df['text'].str.len() > 50]
df.head(5)

,Priority,text
0,P1,Icons needed for actions GI UXW JGS PM We need...
1,P3,README Hit count not reset GET Y JGS AM Set a ...
2,P3,Use styled text in console G S YF DW PM Could ...
3,P3,StringBuffer representation GE BFA JGS PM To b...
4,P2,Feature use toString to display variable value...


# Q2.) Generate a balanced data frame called df_balanced from df by restricting the number of entries in the P3 category to 5000.

In [6]:
# Filter bug reports with priority P3 and sample 5000 rows from it
df_sample_P3 = df[df['Priority'] == 'P3'].sample(n = 5000, random_state=123)

# Create a separate dataframe containing all other bug reports
df_sample_Rest = df[df['Priority'] != 'P3']

# Concatenate the two dataframes to create the new balanced bug reports dataset
df_balanced = pd.concat([df_sample_Rest, df_sample_P3])

# Check the status of the class imbalance
df_balanced['Priority'].value_counts()
df_balanced

,Priority,text
0,P1,Icons needed for actions GI UXW JGS PM We need...
4,P2,Feature use toString to display variable value...
6,P2,Feature Displaying Instruction pointer G A CG ...
7,P2,Feature Locks and Monitors G A ZH DW AM Suppor...
13,P1,Feature navigate from console output to resour...
...,...,...
4955,P3,JavaDoc doclet destination doesnt reflect sele...
15468,P3,Undo doesnt work for Rename Dont try this at h...
28549,P3,Internal compiler error for varags The followi...
30608,P3,compiler fails to detect ambiguous method when...


# Q3.) Lemmatize words and use only 'nav' lemmas for classifications, where 'nav' stands for nouns, pronouns, adjectives, adverbs, and verbs as in FeatureExtraction.

In [7]:
#logic for the feature extraction such as nouns, pronouns, adjectives, adverbs and verbs

nlp = spacy.load("en_core_web_sm")
token = list()
speech = str(df['text'])
doc = nlp(speech)

nav = ("NOUN", "VERB","ADJ","ADV")

count = 0


print("Feature Extraction")
print("--------------------------------------------------------------------------------------------------------------------")
for sent in doc.sents:
    for i in sent:
        if i.pos_ in nav :
            word = re.sub(r"[,.“”]", "", str(i.lemma_))
            word = word.split("\\n")
            print(f"Words :   {word}")
            token.append(word[0])
            count+=1
            
print("\nTotal Count : " , count)

Feature Extraction
--------------------------------------------------------------------------------------------------------------------
Words :   ['icon']
Words :   ['need']
Words :   ['action']
Words :   ['jgs']
Words :   ['pm']
Words :   ['need']
Words :   ['Hit']
Words :   ['count']
Words :   ['reset']
Words :   ['jgs']
Words :   ['am']
Words :   ['use']
Words :   ['style']
Words :   ['text']
Words :   ['console']
Words :   ['representation']
Words :   ['jgs']
Words :   ['pm']
Words :   ['b']
Words :   ['feature']
Words :   ['use']
Words :   ['tostring']
Words :   ['display']
Words :   ['variable']
Words :   ['value']
Words :   ['error']
Words :   ['nullness']
Words :   ['annotation']
Words :   ['development']
Words :   ['face']
Words :   ['lot']
Words :   ['correct']
Words :   ['wrong']
Words :   ['declarat']
Words :   ['show']
Words :   ['fully']
Words :   ['code']
Words :   ['editor']
Words :   ['create']
Words :   ['text']
Words :   ['dtype']
Words :   ['object']

Total Count : 

In [8]:
df_new = df_balanced[['text', 'Priority']]
df_new = df_new.dropna()
print(df_new)

                                                    text Priority
0      Icons needed for actions GI UXW JGS PM We need...       P1
4      Feature use toString to display variable value...       P2
6      Feature Displaying Instruction pointer G A CG ...       P2
7      Feature Locks and Monitors G A ZH DW AM Suppor...       P2
13     Feature navigate from console output to resour...       P1
...                                                  ...      ...
4955   JavaDoc doclet destination doesnt reflect sele...       P3
15468  Undo doesnt work for Rename Dont try this at h...       P3
28549  Internal compiler error for varags The followi...       P3
30608  compiler fails to detect ambiguous method when...       P3
1872   java lang Error Unresolved compilation problem...       P3

[10543 rows x 2 columns]


In [9]:
'''# Loading the balanced dataframe

df_new = df_balanced[['text', 'Priority']]
df_new = df_new.dropna()

# Step 1 - Data Preparation

df_new['text'] = df_new['text'].apply(clean)

# Step 2 - Train-Test Split
X_train_new, X_test_new, Y_train_new, Y_test_new = train_test_split(df_new['text'],
                                                    df_new['Priority'],
                                                    test_size=0.2,
                                                    random_state=42,
                                                    stratify=df_new['Priority'])

print('Number of Training Data : ', X_train_new.shape[0])
print('Number of Test Data : ', X_test_new.shape[0])
'''

"# Loading the balanced dataframe\n\ndf_new = df_balanced[['text', 'Priority']]\ndf_new = df_new.dropna()\n\n# Step 1 - Data Preparation\n\ndf_new['text'] = df_new['text'].apply(clean)\n\n# Step 2 - Train-Test Split\nX_train_new, X_test_new, Y_train_new, Y_test_new = train_test_split(df_new['text'],\n                                                    df_new['Priority'],\n                                                    test_size=0.2,\n                                                    random_state=42,\n                                                    stratify=df_new['Priority'])\n\nprint('Number of Training Data : ', X_train_new.shape[0])\nprint('Number of Test Data : ', X_test_new.shape[0])\n"

In [10]:
'''# Step 3 - Training the Machine Learning model

tfidf_new = TfidfVectorizer(min_df=10, ngram_range=(1, 2), stop_words="english")
X_train_new_tf = tfidf_new.fit_transform(X_train_new)

model_LSVC_new = LinearSVC(random_state=0, tol=1e-5) ### using LinearSVC
model_LSVC_new.fit(X_train_new_tf, Y_train_new)
'''

'# Step 3 - Training the Machine Learning model\n\ntfidf_new = TfidfVectorizer(min_df=10, ngram_range=(1, 2), stop_words="english")\nX_train_new_tf = tfidf_new.fit_transform(X_train_new)\n\nmodel_LSVC_new = LinearSVC(random_state=0, tol=1e-5) ### using LinearSVC\nmodel_LSVC_new.fit(X_train_new_tf, Y_train_new)\n'

In [11]:
'''# Step 4 - Model Evaluation

X_test_new_tf = tfidf_new.transform(X_test_new)
Y_pred_LSVC_new = model_LSVC_new.predict(X_test_new_tf)'''

'# Step 4 - Model Evaluation\n\nX_test_new_tf = tfidf_new.transform(X_test_new)\nY_pred_LSVC_new = model_LSVC_new.predict(X_test_new_tf)'

In [12]:
'''# Vectorization

#tfidf = TfidfVectorizer(min_df = 10, ngram_range=(1,2), stop_words="english")
df_new_tf = tfidf_new.fit_transform(df_new['text']).toarray()

# Cross Validation with 5 batches (folds)

scores = cross_val_score(estimator = model_LSVC_new,
                         X = df_new_tf,
                         y = df_new['Priority'],
                         cv = 5) 

print ("Validation scores for each iteration : ", scores)
print ("Mean value of validation scores : ", scores.mean())
print ("Standard deviation of validation scores : ", scores.std())'''

'# Vectorization\n\n#tfidf = TfidfVectorizer(min_df = 10, ngram_range=(1,2), stop_words="english")\ndf_new_tf = tfidf_new.fit_transform(df_new[\'text\']).toarray()\n\n# Cross Validation with 5 batches (folds)\n\nscores = cross_val_score(estimator = model_LSVC_new,\n                         X = df_new_tf,\n                         y = df_new[\'Priority\'],\n                         cv = 5) \n\nprint ("Validation scores for each iteration : ", scores)\nprint ("Mean value of validation scores : ", scores.mean())\nprint ("Standard deviation of validation scores : ", scores.std())'

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import plot_confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.svm import LinearSVC
from sklearn.dummy import DummyClassifier


training_pipeline = Pipeline(
    steps=[('tfidf', TfidfVectorizer(stop_words="english")), 
           ('LinearSVC', LinearSVC(random_state=21, tol=1e-5))])

grid_param = [{
'tfidf__min_df': [5, 10],
'tfidf__ngram_range': [(1, 3), (1, 6)],
'LinearSVC__penalty': ['l2'],
'LinearSVC__loss': ['hinge'],
'LinearSVC__max_iter': [10000]
}, {
'tfidf__min_df': [5, 10],
'tfidf__ngram_range': [(1, 3), (1, 6)],
'LinearSVC__C': [1, 10],
'LinearSVC__tol': [1e-2, 1e-3]
}]


gridSearchProcessor = GridSearchCV(estimator=training_pipeline,
                                   param_grid=grid_param,
                                   cv=5)

gridSearchProcessor.fit(df_balanced['text'], df_balanced['Priority'])


print("\n")
print("Grid Search Results : ")
print("------------------------------------------------------------------------------")
best_params = gridSearchProcessor.best_params_
print("Best_Parameters : ", best_params)

best_result = gridSearchProcessor.best_score_
print("\n")
print("Best_Result : ", best_result)




Grid Search Results : 
------------------------------------------------------------------------------
Best_Parameters :  {'LinearSVC__loss': 'hinge', 'LinearSVC__max_iter': 10000, 'LinearSVC__penalty': 'l2', 'tfidf__min_df': 5, 'tfidf__ngram_range': (1, 6)}


Best_Result :  0.5034536184041827


In [14]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import plot_confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.dummy import DummyClassifier

training_pipeline = Pipeline(
    steps=[('tfidf', TfidfVectorizer(stop_words="english")), 
           ('SVC', SVC(random_state=21, tol=1e-5))])

tuned_parameters = [{
    'tfidf__min_df': [5, 10],
    'tfidf__ngram_range': [(1, 3), (1, 6)],
    'SVC__C': [1, 10, 100, 1000],
    'SVC__kernel': ['rbf', 'linear'], 
    'SVC__gamma': [1e-3, 1e-4],
    "SVC__tol": [1e-2, 1e-10]
}]

gridSearchProcessor = GridSearchCV(estimator=training_pipeline,
                                   param_grid=tuned_parameters,
                                   cv=2, n_jobs=-1)

gridSearchProcessor.fit(df_balanced['text'], df_balanced['Priority'])
print()
print("Grid Search Results : ")
print("------------------------------------------------------------------------------")
best_params_svc = gridSearchProcessor.best_params_
print("Best_Parameters : ", best_params_svc)

best_result_svc = gridSearchProcessor.best_score_
print("Best_Result : ", best_result_svc)


Grid Search Results : 
------------------------------------------------------------------------------
Best_Parameters :  {'SVC__C': 1, 'SVC__gamma': 0.001, 'SVC__kernel': 'rbf', 'SVC__tol': 0.01, 'tfidf__min_df': 5, 'tfidf__ngram_range': (1, 3)}
Best_Result :  0.4742483206850321


In [15]:
# Loading the balanced dataframe

df_new = df_balanced[['text', 'Priority']]
df_new = df_new.dropna()

# Step 1 - Data Preparation

df_new['text'] = df_new['text'].apply(clean)

# Step 2 - Train-Test Split
X_train, X_test, Y_train, Y_test = train_test_split(df_new['text'],
                                                    df_new['Priority'],
                                                    test_size=0.2, ### 80-20 train-test split
                                                    random_state=42,
                                                    stratify=df_new['Priority'])

print('Number of Training Data : ', X_train.shape[0])
print('Number of Test Data : ', X_test.shape[0])

Number of Training Data :  8434
Number of Test Data :  2109


In [16]:
from sklearn.svm import LinearSVC
from sklearn.datasets import load_iris
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(min_df = 10, ngram_range=(1,2), stop_words="english") 
### from sklearn
### min_df = 10 means that terms appearing in strickly less than 10 documents are ignored
X_train_tf = tfidf.fit_transform(X_train)

model_LSVC = LinearSVC(random_state=0, tol=1e-5) 
### tol: Tolerance for stopping criteria; 1e-5 = e^{-5} = exp(-5)
model_LSVC.fit(X_train_tf, Y_train)

X_test_tf = tfidf.transform(X_test)
Y_pred_LSVC = model_LSVC.predict(X_test_tf) ### evaluation
print("\n")
print ('Accuracy_Score : ', accuracy_score(Y_test, Y_pred_LSVC))



Accuracy_Score :  0.5030820293978189


In [17]:
from sklearn.datasets import load_iris
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

tfidf = TfidfVectorizer(min_df = 10, ngram_range=(1,2), stop_words="english") 
### from sklearn
### min_df = 10 means that terms appearing in strickly less than 10 documents are ignored
X_train_tf = tfidf.fit_transform(X_train)

model_SVC = SVC(random_state=0, tol=1e-5) 
### tol: Tolerance for stopping criteria; 1e-5 = e^{-5} = exp(-5)
model_SVC.fit(X_train_tf, Y_train)

X_test_tf = tfidf.transform(X_test)
Y_pred_SVC = model_SVC.predict(X_test_tf) ### evaluation
print("\n")
print ('Accuracy_Score : ', accuracy_score(Y_test, Y_pred_SVC))



Accuracy_Score :  0.5367472735893789


In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

LSVC_tfidf = TfidfVectorizer(min_df=best_params['tfidf__min_df'],
                           ngram_range=best_params['tfidf__ngram_range'],
                           stop_words='english')
LSVC_X_train_tf = LSVC_tfidf.fit_transform(X_train)


best_modal_lsvc=LinearSVC(random_state=21,penalty = 'l2',max_iter = 10000,
                          loss= 'hinge')
best_modal_lsvc.fit(LSVC_X_train_tf,Y_train)





SVC_tfidf = TfidfVectorizer(min_df = best_params['tfidf__min_df'],
                           ngram_range = best_params['tfidf__ngram_range'],
                           stop_words = 'english')

SVC_X_train_tf = SVC_tfidf.fit_transform(X_train)
best_modal_svc = SVC(random_state=21)
best_modal_svc.fit(SVC_X_train_tf,Y_train)

SVC(random_state=21)

In [19]:
LSVC_X_test_tf=LSVC_tfidf.transform(X_test)
Y_pred_LSVC=best_modal_lsvc.predict(LSVC_X_test_tf)
print ('\nLinearSVC_Accuracy_Score : ', accuracy_score(Y_test, Y_pred_LSVC))
print("-------------------------------------------------------------------------------------------------------")
print (classification_report(Y_test, Y_pred_LSVC))



SVC_X_test_tf=SVC_tfidf.transform(X_test)
Y_pred_SVC=best_modal_svc.predict(SVC_X_test_tf)
print ('\nSVC_Accuracy_Score - ', accuracy_score(Y_test, Y_pred_SVC))
print("-------------------------------------------------------------------------------------------------------")
print (classification_report(Y_test, Y_pred_SVC))


LinearSVC_Accuracy_Score :  0.5376955903271693
-------------------------------------------------------------------------------------------------------
              precision    recall  f1-score   support

          P1       0.48      0.25      0.32       224
          P2       0.45      0.32      0.37       607
          P3       0.57      0.82      0.67      1000
          P4       0.50      0.29      0.37       228
          P5       0.00      0.00      0.00        50

    accuracy                           0.54      2109
   macro avg       0.40      0.34      0.35      2109
weighted avg       0.51      0.54      0.50      2109


SVC_Accuracy_Score -  0.5339023233760076
-------------------------------------------------------------------------------------------------------
              precision    recall  f1-score   support

          P1       0.61      0.16      0.25       224
          P2       0.45      0.26      0.33       607
          P3       0.54      0.90      0.68      1

/Users/sagar/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sagar/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sagar/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [20]:
linear_modal_SVC = SVC(C = best_params_svc['SVC__C'],
                       kernel = 'linear',gamma=best_params_svc['SVC__gamma'],tol=best_params_svc['SVC__tol'],probability=True,random_state=42)

linear_modal_SVC.fit(SVC_X_train_tf, Y_train)

SVC_X_test_tf = SVC_tfidf.transform(X_test)

Y_pred_Linear_Kernal_SVC= linear_modal_SVC.predict(SVC_X_test_tf)
print()
print ('\nAccuracy Score - ', accuracy_score(Y_test, Y_pred_Linear_Kernal_SVC))
print (classification_report(Y_test, Y_pred_Linear_Kernal_SVC))



Accuracy Score -  0.5400663821716454
              precision    recall  f1-score   support

          P1       0.52      0.20      0.29       224
          P2       0.44      0.34      0.39       607
          P3       0.57      0.83      0.68      1000
          P4       0.58      0.23      0.33       228
          P5       0.00      0.00      0.00        50

    accuracy                           0.54      2109
   macro avg       0.42      0.32      0.34      2109
weighted avg       0.52      0.54      0.50      2109



/Users/sagar/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sagar/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sagar/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [21]:
rbf_modal_SVC=SVC(C = best_params_svc['SVC__C'],kernel = 'rbf',gamma=best_params_svc['SVC__gamma'],tol=best_params_svc['SVC__tol'],probability=True,random_state=42)
rbf_modal_SVC.fit(SVC_X_train_tf, Y_train)

SVC_X_test_tf=SVC_tfidf.transform(X_test)

Y_pred_rbf_Kernal_SVC= rbf_modal_SVC.predict(SVC_X_test_tf)
print()
print ('\nAccuracy Score - ', accuracy_score(Y_test, Y_pred_rbf_Kernal_SVC))
print (classification_report(Y_test, Y_pred_rbf_Kernal_SVC))



Accuracy Score -  0.474158368895211
              precision    recall  f1-score   support

          P1       0.00      0.00      0.00       224
          P2       0.00      0.00      0.00       607
          P3       0.47      1.00      0.64      1000
          P4       0.00      0.00      0.00       228
          P5       0.00      0.00      0.00        50

    accuracy                           0.47      2109
   macro avg       0.09      0.20      0.13      2109
weighted avg       0.22      0.47      0.31      2109



/Users/sagar/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sagar/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sagar/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [22]:
Y_pred_LSVC=best_modal_lsvc.predict(LSVC_X_test_tf)
LSVC_results= pd.DataFrame({'text':X_test.values,'actual':Y_test.values,'predicted':Y_pred_LSVC})
print(LSVC_results)

Y_pred_SVC=model_SVC.predict(X_test_tf)
SVC_results=pd.DataFrame({'text':X_test.values,'actual':Y_test.values,'predicted':Y_pred_SVC})
print(SVC_results)

Y_pred_Linear_Kernal_SVC=linear_modal_SVC.predict(SVC_X_test_tf)
Linear_SVC_results=pd.DataFrame({'text':X_test.values,'actual':Y_test.values,'predicted':Y_pred_Linear_Kernal_SVC})

Y_pred_rbf_Kernal_SVC=rbf_modal_SVC.predict(SVC_X_test_tf)
rbf_SVC_results=pd.DataFrame({'text':X_test.values,'actual':Y_test.values,'predicted':Y_pred_rbf_Kernal_SVC})

                                                   text actual predicted
0     Using backspace in Code Assist causes menu to ...     P3        P3
1     Unable to run debug a program Hi with the newe...     P3        P3
2     Java compiler doesnt properly flag invalid pro...     P3        P3
3     Incorrect hover description The description sh...     P3        P2
4     Enhance additional info popup JavaDoc window t...     P2        P3
...                                                 ...    ...       ...
2104  Replace with latest from repository closes Jav...     P3        P2
2105  package names truncated in compilation dialog ...     P3        P3
2106  Configure access rules should use the new conf...     P3        P3
2107  JDTCompilerAdapter doesnt use user preference ...     P3        P3
2108  ObjectActionDelegate hierarchy memory leak Thi...     P2        P1

[2109 rows x 3 columns]
                                                   text actual predicted
0     Using backspace in C

In [23]:
linear_modal_SVC.coef_

# coef = linear_modal_SVC.coef_[8].A[0]
# vocabulary_positions = coef.argsort()
# vocabulary = tfidf.get_feature_names()

# top_words = 5
# top_positive_coef = vocabulary_positions[-top_words:].tolist()
# top_negative_coef = vocabulary_positions[:top_words].tolist()
# print(top_positive_coef)
# print(top_negative_coef)

# core_ui = pd.DataFrame([[vocabulary[c], coef[c]] for c in top_positive_coef + top_negative_coef], 
#                           columns=["feature", "coefficient"]).sort_values("coefficient")

coef = linear_modal_SVC.coef_[9].A[0]
print(coef)
vocabulary_positions = coef.argsort()
vocabulary = tfidf.get_feature_names()
top_words = 5
top_positive_coef = vocabulary_positions[-top_words:].tolist()
top_negative_coef = vocabulary_positions[:top_words].tolist()
print(top_positive_coef)
print(top_negative_coef)
print(top_positive_coef + top_negative_coef)
core_ui = pd.DataFrame([[vocabulary[c], coef[c]] for c in range(len(top_positive_coef+top_negative_coef))], 
                          columns=["feature", "coefficient"]).sort_values("coefficient")
core_ui.set_index("feature")

[0.         0.         0.         ... 0.         0.02349588 0.        ]
[33, 49922, 34882, 32096, 30944]
[43704, 50894, 16021, 31325, 28002]
[33, 49922, 34882, 32096, 30944, 43704, 50894, 16021, 31325, 28002]


/Users/sagar/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,coefficient
feature,
able select,-0.077287
aa,0.000000
ab,0.000000
abc,0.000000
ability,0.000000
able,0.000000
able reproduce,0.000000
able specify,0.021652
able set,0.338433


In [26]:
from sklearn.pipeline import make_pipeline
pipeline = make_pipeline(tfidf, rbf_modal_SVC)
result = pd.DataFrame({ 'text': X_test.values, 'actual': Y_test.values, 'predicted': Y_pred_SVC })
class_names = ["text", "predicted"]
prob = rbf_modal_SVC.predict_proba(SVC_X_test_tf)
prob_svc = prob
# new dataframe for explainable results
er = result.copy().reset_index() ### er stands for "explain result"
explain_result = er
er['max_probability'] = er[class_names].max(axis=1)
class_names = ["text", "predict"]
from lime.lime_text import LimeTextExplainer
explainer = LimeTextExplainer(class_names=class_names)
er[er["predicted"] != er["actual"]].head(10)

,index,text,actual,predicted,max_probability
3,3,Incorrect hover description The description sh...,P3,P2,P2
4,4,Enhance additional info popup JavaDoc window t...,P2,P3,P3
5,5,Code assist should not just beep suggest to sh...,P2,P3,P3
6,6,Override Methods doesnt handle unicodes Using ...,P4,P3,P3
9,9,ClassCastException expanding inspect expressio...,P1,P3,P3
13,13,NPE opening editor when some plugins disabled ...,P1,P2,P2
14,14,Add from local history formatting GF GP TM SDK...,P1,P2,P2
15,15,unable to retrieve system properties Olivier r...,P2,P3,unable to retrieve system properties Olivier r...
16,16,preference page strings missing in latest as i...,P1,P3,preference page strings missing in latest as i...
18,18,Extract Interface superfluos final argument mo...,P4,P3,P3
